<a href="https://colab.research.google.com/github/BenyaminZojaji/Deep_Learning/blob/main/Face%20Recognition/faceRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPool2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wandb
from wandb.keras import WandbCallback

### wandb configuration

In [31]:
batch_size = 32
width = height = 224

In [32]:
config = {
              "learning_rate": 0.001,
              "epochs": 25,
              "batch_size": 32,
              "log_step": 200,
              "val_log_step": 50
           }
run = wandb.init(project='faceRecognition', config=config)
config = wandb.config

wandb: Currently logged in as: benyaminzojaji (use `wandb login --relogin` to force relogin)


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load Dataset

In [33]:
dataset_path = '/content/drive/MyDrive/7-7 dataset'

idg = ImageDataGenerator(
    rescale = 1.0/255.0,
    horizontal_flip=True,
    brightness_range=(0.8,1.2),
    zoom_range=0.2,
    validation_split=0.2
)

train_data = idg.flow_from_directory(
    dataset_path,
    target_size=(width, height),
    class_mode='categorical',
    batch_size=batch_size,
    subset='training'
)

val_data = idg.flow_from_directory(
    dataset_path,
    target_size=(width, height),
    class_mode='categorical',
    batch_size=batch_size,
    subset='validation'
)

Found 1090 images belonging to 14 classes.
Found 268 images belonging to 14 classes.


In [41]:
class MyModel(Model):
  def __init__(self, number_of_classes):
    super().__init__()
    
    self.dense_1 = Dense(128, activation='relu')
    self.dense_2 = Dense(256, activation='relu')
    self.dense_3 = Dense(number_of_classes, activation='softmax')
    self.conv2d_1 = Conv2D(32, (3, 3), activation='relu', input_shape = (width, height, 3))
    self.conv2d_2 = Conv2D(64, (5, 5), activation='relu')
    self.conv2d_3 = Conv2D(128, (5, 5), activation='relu')
    self.conv2d_4 = Conv2D(256, (3, 3), activation='relu')
    self.maxpool = MaxPool2D()
    self.flatten = Flatten()
    self.dropout1 = Dropout(0.3)

  def call(self, x):
    x = self.conv2d_1(x)
    #x = self.conv2d_2(x)
    x = self.maxpool(x)
    x = self.conv2d_3(x)
    #x = self.conv2d_4(x)
    x = self.flatten(x)
    x = self.dropout1(x)
    #x = self.dense_1(x)
    #x = self.dense_2(x)
    out = self.dense_3(x)

    return out

In [35]:
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer =  tf.keras.optimizers.Adam(config.learning_rate)

In [36]:
train_loss = tf.keras.metrics.MeanAbsoluteError()
test_loss = tf.keras.metrics.MeanAbsoluteError()
train_accuracy = tf.keras.metrics.CategoricalAccuracy()
test_accuracy = tf.keras.metrics.CategoricalAccuracy()

In [42]:
model = MyModel(14)

In [43]:
def train_step(images, labels):
  with tf.GradientTape() as gTape:
    predictions = model(images, training = True)
    loss= loss_function(labels, predictions)
    train_loss(labels, predictions)
    train_accuracy(labels, predictions)
  
  # calculate gradients
  gradients = gTape.gradient(loss, model.trainable_variables)

  # update weights
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [44]:
def test_step(images, labels):
  predictions = model(images, training = False)
  loss= loss_function(labels, predictions)
  test_accuracy(labels, predictions)
  test_loss(labels, predictions)

In [45]:
def train():
    epochs = 15

    for epoch in range(epochs):
        train_loss.reset_state()
        test_loss.reset_state()
        train_accuracy.reset_state()
        test_accuracy.reset_state()
        print('epoch:', epoch + 1)

        for index, (images, labels) in enumerate(tqdm(train_data)):
            train_step(images, labels)
            if len(train_data) <= index:
              break

        for index, (images, labels) in enumerate(tqdm(val_data)):
            test_step(images, labels)
            if len(val_data) <= index:
              break
        
        print('accuracy:', train_accuracy.result())
        print('val accuracy:', test_accuracy.result())
        print('loss:', train_loss.result())
        print('val loss:', test_loss.result())

        wandb.log({'epochs': epoch,
            'train_loss': np.mean(train_loss.result()),
            'train_accuracy': float(train_accuracy.result()), 
            'test_loss': np.mean(test_loss.result()),
            'test_accuracy': float(test_accuracy.result())
            })

In [46]:
train()
run.finish()

epoch: 1


100%|██████████| 9/9 [00:09<00:00,  1.10s/it]


accuracy: tf.Tensor(0.27005348, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.41666666, shape=(), dtype=float32)
loss: tf.Tensor(0.11897589, shape=(), dtype=float32)
val loss: tf.Tensor(0.09894117, shape=(), dtype=float32)
epoch: 2


100%|██████████| 9/9 [00:09<00:00,  1.10s/it]


accuracy: tf.Tensor(0.62923354, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.5833333, shape=(), dtype=float32)
loss: tf.Tensor(0.076102555, shape=(), dtype=float32)
val loss: tf.Tensor(0.0710448, shape=(), dtype=float32)
epoch: 3


...

100%|██████████| 9/9 [00:09<00:00,  1.10s/it]


accuracy: tf.Tensor(0.94741535, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.78, shape=(), dtype=float32)
loss: tf.Tensor(0.010276248, shape=(), dtype=float32)
val loss: tf.Tensor(0.03607814, shape=(), dtype=float32)
epoch: 15


100%|██████████| 9/9 [00:09<00:00,  1.10s/it]


accuracy: tf.Tensor(0.96880573, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.84, shape=(), dtype=float32)
loss: tf.Tensor(0.00738422, shape=(), dtype=float32)
val loss: tf.Tensor(0.025144214, shape=(), dtype=float32)



epochs,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
test_accuracy,▁▄▅▇▆▇▇▇▇▇▇█▇▇█
test_loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▁
train_accuracy,▁▅▆▇▇▇▇████████
train_loss,█▅▄▃▂▂▂▂▂▂▁▁▁▁▁
epochs,14
test_accuracy,0.84
test_loss,0.02514
train_accuracy,0.96881
train_loss,0.00738


In [25]:
model.save_weights('faceRecognition', save_format='HDF5')